<a href="https://colab.research.google.com/github/RanjuLG/sports-player-tracking/blob/main/sports_player_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!nvidia-smi

Wed Nov 12 18:29:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.0 MB/s eta 0:00:00


In [4]:
# Define Project Paths
BASE_DRIVE_PATH = '/content/gdrive/MyDrive/SC549_Assignment_3/inputs'


# List the content of the BASE_DRIVE_PATH
!ls -F "{BASE_DRIVE_PATH}"


cricket_1.mp4  football_1.mp4  football_3.mp4  rugby_2.mp4
cricket_2.mp4  football_2.mp4  rugby_1.mp4     rugby_3.mp4


In [5]:
#Define Project Paths and Find Videos
import os

BASE_DRIVE_PATH = '/content/gdrive/MyDrive/SC549_Assignment_3'

INPUT_FOLDER_NAME = 'inputs'

# Auto-create paths
INPUT_VIDEO_DIR = os.path.join(BASE_DRIVE_PATH, INPUT_FOLDER_NAME)
RESULTS_OUTPUT_DIR = os.path.join(BASE_DRIVE_PATH, 'outputs')

# Create the main output directory if it doesn't exist
os.makedirs(RESULTS_OUTPUT_DIR, exist_ok=True)

video_extensions = ('.mp4')

try:
    all_files = os.listdir(INPUT_VIDEO_DIR)
    video_files_to_process = [f for f in all_files if f.lower().endswith(video_extensions)]
except FileNotFoundError:
    print(f"ERROR: Input directory not found at {INPUT_VIDEO_DIR}")
    print("Please check your 'BASE_DRIVE_PATH' and 'INPUT_FOLDER_NAME' variables.")
    video_files_to_process = []

if not video_files_to_process:
    print(f"No video files found in {INPUT_VIDEO_DIR}.")
else:
    print(f"Successfully found {len(video_files_to_process)} videos to process:")
    for vid_name in video_files_to_process:
        print(f"- {vid_name}")

Successfully found 8 videos to process:
- rugby_1.mp4
- rugby_2.mp4
- rugby_3.mp4
- football_1.mp4
- football_3.mp4
- cricket_1.mp4
- cricket_2.mp4
- football_2.mp4


In [6]:
#Load Models
from ultralytics import YOLO
model_detect = YOLO('yolov8n.pt')
model_pose = YOLO('yolov8n-pose.pt')

print("Models loaded successfully.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Models loaded successfully.


In [7]:
# Loop and Process All Videos

if not video_files_to_process:
    print("Skipping processing as no video files were found.")
else:
    print(f"\n Starting Batch Processing for {len(video_files_to_process)} Videos")

    for i, video_name in enumerate(video_files_to_process):
        print(f"\n ")
        print(f"  Processing Video {i+1} of {len(video_files_to_process)}: {video_name}")

        # Define the full path for the current input video
        current_video_input_path = os.path.join(INPUT_VIDEO_DIR, video_name)

        # Task 1: Player Detection & Tracking ---
        print(f"[Task 1/2] Running detection & tracking on {video_name}...")
        model_detect.track(
            source=current_video_input_path,
            classes=0, # Only 'person'
            conf=0.3,
            save=True,
            project=RESULTS_OUTPUT_DIR, # Main output folder
            name=f'{video_name}_detect_run' # Unique subfolder for this video's results
        )
        print(f"[Task 1/2] Finished detection for {video_name}.")


        # Task 2: Keypoint Detection
        print(f"[Task 2/2] Running keypoint (pose) detection on {video_name}...")
        model_pose.predict(
            source=current_video_input_path,
            conf=0.3,
            save=True,
            project=RESULTS_OUTPUT_DIR, # Main output folder
            name=f'{video_name}_pose_run' # Unique subfolder for this video's results
        )
        print(f"[Task 2/2] Finished pose estimation for {video_name}.")

    print("\n\n All Videos Processed Successfully!")

Streaming output truncated to the last 5000 lines.
video 1/1 (frame 180/181) /content/gdrive/MyDrive/SC549_Assignment_3/inputs/rugby_1.mp4: 384x640 13 persons, 7.5ms
video 1/1 (frame 181/181) /content/gdrive/MyDrive/SC549_Assignment_3/inputs/rugby_1.mp4: 384x640 13 persons, 6.5ms
Speed: 3.8ms preprocess, 10.8ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/gdrive/MyDrive/SC549_Assignment_3/outputs/rugby_1.mp4_detect_run2
[Task 1/2] Finished detection for rugby_1.mp4.
[Task 2/2] Running keypoint (pose) detection on rugby_1.mp4...

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r